In [1]:
""" Convolutional Neural Network for MNIST dataset classification task.
References:
    Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. "Gradient-based
    learning applied to document recognition." Proceedings of the IEEE,
    86(11):2278-2324, November 1998.
Links:
    [MNIST Dataset] http://yann.lecun.com/exdb/mnist/
Source:
    https://github.com/tflearn/tflearn/blob/master/examples/images/convnet_mnist.py
"""

from __future__ import division, print_function, absolute_import

import tflearn
import tensorflow as tf
from tflearn.layers.core import input_data, dropout, fully_connected, flatten
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression
from tflearn.activations import relu, softmax
from tflearn.layers.normalization import batch_normalization

# Data loading and preprocessing
import tflearn.datasets.mnist as mnist
X, Y, testX, testY = mnist.load_data(one_hot=True)
X = X.reshape([-1, 28, 28, 1])
testX = testX.reshape([-1, 28, 28, 1])

# Choose binarization method
binarize = 'full'
name = 'LeNet-full'

def conv(network, filters, kernel, strides, activation):
    network = batch_normalization(network)
    network = conv_2d(network, filters, kernel, strides=strides, activation=activation, binarize=binarize)
    return network

def fc(network, n_units, activation):
    network = batch_normalization(network)
    network = fully_connected(network, n_units, activation=activation, binarize=binarize)
    return network
    
# Building convolutional network
network = input_data(shape=[None, 28, 28, 1], name='input')
network = conv(network, 32, 3, 1, 'relu')
network = max_pool_2d(network, 2)
network = conv(network, 64, 3, 1, 'relu')
network = max_pool_2d(network, 2)
network = fc(network, 1024, 'tanh')
network = dropout(network, 0.5)
network = fc(network, 10, None)
network = softmax(network)
network = regression(network, optimizer='adam', learning_rate=0.001,
                     loss='categorical_crossentropy', name='target')

# Training
model = tflearn.DNN(network, tensorboard_verbose=0)
model.fit({'input': X}, {'target': Y}, n_epoch=2, validation_set=({'input': testX}, {'target': testY}),
           show_metric=True, run_id=name, batch_size=50)

Training Step: 2199  | total loss: 0.21897 | time: 42.008s
| Adam | epoch: 002 | loss: 0.21897 - acc: 0.9360 -- iter: 54950/55000
Training Step: 2200  | total loss: 0.21378 | time: 44.192s
| Adam | epoch: 002 | loss: 0.21378 - acc: 0.9364 | val_loss: 0.20033 - val_acc: 0.9416 -- iter: 55000/55000
--
